# Thermal and Electrical Analysis of 1N4148 Diode

This notebook evaluates forward conduction, thermal resistance, and junction temperature analysis using measured and datasheet data of the 1N4148 PN diode.

In [8]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from IPython.display import Markdown, display

# Constants
k = 1.380649e-23  # Boltzmann constant (J/K)
q = 1.602176634e-19  # Elementary charge (C)
T_amb = 25 + 273.15  # Ambient temperature in Kelvin
Vf = 0.839  # Forward voltage in Volts
If = 0.05  # Forward current in Amps
Rs = 50  # Series resistance in Ohms
Rth_j_a = 350  # Thermal resistance junction to ambient in K/W
Rth_j_tp = 240  # Thermal resistance junction to tie-point in K/W
Ha = 20  # Heat transfer coefficient in W/m^2K
length = 3.4e-3  # Length in meters
diameter = 1.75e-3  # Diameter in meters

# Surface area of cylindrical glass body (neglecting leads)
A_surface = np.pi * diameter * length

# Power dissipation
P = Vf * If

# Junction temperature rise
Tj_conv = T_amb + P * Rth_j_a
Tj_lead = T_amb + P * Rth_j_tp
Tj_Ha = T_amb + P / (Ha * A_surface)

# Ideality factor
Is = 5.84e-9
n = (Vf * q) / (k * T_amb * np.log(If / Is))

# Display
results = f"""
### Thermal Analysis of 1N4148 Diode

**Power Dissipation:**  
$P = V_F * I_F = {Vf:.3f} * {If*1e3:.1f} = {P*1e3:.2f}\ {{mW}}$

**Junction Temperature Estimates:**  
- $R_{{\text{{th-j-a}}}} = 350$ K/W → $T_j = {Tj_conv - 273.15:.2f}^\circ$C  
- $R_{{\text{{th-j-tp}}}} = 240$ K/W → $T_j = {Tj_lead - 273.15:.2f}^\circ$C  
- $h = {Ha} $ W/m²K, $A = {A_surface*1e6:.2f}$ mm² → $T_j = {Tj_Ha - 273.15:.2f}^\circ$C  

---

### Ideality Factor Estimate  
$n = {n:.3f}$
"""
display(Markdown(results))



### Thermal Analysis of 1N4148 Diode

**Power Dissipation:**  
$P = V_F * I_F = 0.839 * 50.0 = 41.95\ {mW}$

**Junction Temperature Estimates:**  
- $R_{	ext{th-j-a}} = 350$ K/W → $T_j = 39.68^\circ$C  
- $R_{	ext{th-j-tp}} = 240$ K/W → $T_j = 35.07^\circ$C  
- $h = 20 $ W/m²K, $A = 18.69$ mm² → $T_j = 137.21^\circ$C  

---

### Ideality Factor Estimate  
$n = 2.046$


# Key Conclusions and Inferences for 1N4148 Diode

---

## 1. Diode's Exponential I–V Behavior

The Shockley equation models the diode forward current as:

$$
I_F = I_S \left(e^{\frac{qV_F}{nkT}} - 1\right)
$$

- The forward current rises exponentially with increasing $V_F$.
- On a semilog scale, the I–V curve becomes linear.
- The extracted ideality factor $n$ from SPICE or datasheet closely matches expected values.

---

## 2. Temperature Impact

- Forward voltage $V_F$ decreases as junction temperature $T_j$ increases.
- Based on datasheet Fig. 1:

| $T_j$ (°C) | $V_F$ @ $I_F = 10$ mA |
|-----------|------------------------|
| 25        | ~0.72 V                |
| 75        | ~0.65 V                |
| 150       | ~0.58 V                |

- Approximate temperature coefficient:

$$
\frac{dV_F}{dT} \approx -1.12 \, \text{mV}/^\circ\text{C}
$$

This confirms the ~–1 to –2 mV/°C slope expected in silicon diodes.

---

## 3. Threshold Region

- The diode starts conducting significantly above $V_F \approx 0.6 \, \text{V}$.
- For a current $I_F = 50 \, \text{mA}$, $V_F$ reaches ~839 mV.
- Below 0.4 V, current is negligible (μA to nA range).

---

## 4. Diode Ideality Factor Check

Rewriting the Shockley equation in logarithmic form:

$$
\log_{10} I_F = \log_{10} I_S + \frac{qV_F}{n k T \ln 10}
$$

This gives:

$$
\frac{dV_F}{d(\log_{10} I_F)} = \frac{n k T \ln 10}{q} \approx 59 \, \text{mV} \times n
$$

From datasheet graphs:
- For each decade increase in $I_F$, $\Delta V_F \approx 60 - 70$ mV
- This implies $n \approx 1.0 - 1.2$ — close to ideal!

---

## 5. Reverse Use – Estimate $T_j$ from $V_F$

- If you know $V_F$ and $I_F$, you can estimate junction temperature $T_j$ using datasheet graphs or Shockley equation.
- This makes the diode a **low-cost thermal sensor** under steady-state conditions.

As discussed by Ocaya [1], a P–N diode's forward voltage drop is sensitive to temperature, making it useful as a low-cost temperature sensor.

[1]: R. O. Ocaya, *An experiment to profile the voltage, current and temperature behaviour of a P–N diode*, *Eur. J. Phys.*, vol. 27, no. 3, pp. 651–660, 2006. https://doi.org/10.1088/0143-0807/27/3/015

---

**SPICE Validation:**
- Simulation at 50 mA gives $V_F \approx 839$ mV.
- This aligns well with both datasheet expectations and model accuracy.

---

## Reference Datasheet Values

- $V_{F, max} = 1.0$ V at $I = 100$ mA
- $I_F$ (continuous) = 200 mA max
- $P_{tot}$ = 500 mW @ 25°C
- $T_j$ max = 200°C
- $R_{th-j-a} = 350$ K/W, $R_{th-j-tp} = 240$ K/W

Use resistor to limit current for 5V supply:  
$R = \frac{V_{supply} - V_F}{I_F} ⇒ R_{min} = \frac{5 - 0.839}{0.05} ≈ 83.2\ \Omega$

